In [3]:
# Import packages.
import numpy as np
import pandas as pd
import nltk
import nltk.corpus
import gzip
import json
import re
from nltk.corpus import wordnet

#### Build function to clean text with test data.

In [4]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)
    
    
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF("../../../Downloads/AMAZON_FASHION_5.json.gz")

In [5]:
df.head()

overall  verified  reviewTime     reviewerID        asin  \
0      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   
1      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   
2      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   
3      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   
4      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   

                                               style reviewerName  \
0   {'Size:': ' Big Boys', 'Color:': ' Blue/Orange'}     Tonya B.   
1  {'Size:': ' Big Boys', 'Color:': ' Black (3746...     Tonya B.   
2  {'Size:': ' Big Boys', 'Color:': ' Blue/Gray L...     Tonya B.   
3  {'Size:': ' Big Boys', 'Color:': ' Blue (37867...     Tonya B.   
4     {'Size:': ' Big Boys', 'Color:': ' Blue/Pink'}     Tonya B.   

                 reviewText     summary  unixReviewTime vote image  
0  Great product and price!  Five Stars      1441324800  NaN   NaN  
1  Great product and price!  Five Stars      1441324800  NaN   NaN  
2  Great product and price!  Five Stars      1441324800  NaN   NaN  
3  Great product and price!  Five Stars      1441324800  NaN   NaN  
4  Great product and price!  Five Stars      1441324800  NaN   NaN

In [6]:
np.shape(df)

(3176, 12)

In [7]:
# Drop duplicate reviews
df_nodup = df.drop_duplicates(subset = ['reviewText'])

In [8]:
def tokenize_text(doc):
    """
    Input: A string of words.
    Output: List of tokenized words that are all lowercase.
    """

    # Tokenize and make lowercase.
    words = nltk.word_tokenize(doc)
    words = [w.lower() for w in words]
    
    return words


def wordnet_pos(tag):
    """
    Map a Brown POS tag to a WordNet POS tag. This is for lemmatization.
    """
    
    table = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}
    
    # Default to a noun.
    return table.get(tag[0], wordnet.NOUN)


def lemmatize_text(words):
    """
    Input: A list of tokenized words.
    Output: A list of tokenized words that are lemmatized.
    """
    
    lemmatizer = nltk.WordNetLemmatizer()
    word_tags = nltk.pos_tag(words)
    words = [lemmatizer.lemmatize(w, wordnet_pos(t)) for (w, t) in word_tags]
    
    return words


def remove_stopwords(words):
    """
    Input: A list of tokenized words.
    Output: A list of tokenized words that have stopwords removed.
    """
    
    stopwords = nltk.corpus.stopwords.words("english")
    words = [w for w in words if w not in stopwords]
    
    return words

def clean_text(doc): 
    """
    Input: A string of words.
    Output: A string of words that has been lemmatized, has the stopwords removed, and has the puncuation removed.
    """
    
    words = re.sub("< ?/?[a-z]+ ?>|\n", "", doc)
    words = tokenize_text(words)
    words = lemmatize_text(words)
    words = remove_stopwords(words)
    doc = [w for w in words if w.isalnum()]
    doc = ' '.join(doc)
    
    return doc

def clean_df(df):
    """
    Input: A dataframe with a column of reviews called 'reviewText'.
    Output: The same dataframe as the input, but with an extra column called 'text' which has the 
            cleaned 'reviewText'.
    """
    
    text = df['reviewText']
    df_clean = df.copy()
    df_clean['text'] = [clean_text(str(i)) for i in text]

    return df_clean

In [9]:
print(df['reviewText'][10])
print(clean_text(df['reviewText'][10]))

Relieved my Plantar Fascitis for 3 Days. Then the unbearable pain returned in full force. These were recommended by my Podiatrist.
relieve plantar fascitis 3 day unbearable pain return full force recommend podiatrist


In [10]:
print(df['reviewText'][300])
print(clean_text(df['reviewText'][300]))

Love these sneakers. Light weight and comfortable even without socks.
love sneaker light weight comfortable even without sock


In [11]:
# Test the function
clean_df(df_nodup)[['reviewText', 'text']].head()

reviewText  \
0                           Great product and price!   
5      Waaay too small. Will use for futur children!   
6                    Stays vibrant after many washes   
8  My son really likes the pink. Ones which I was...   
9        Waaay too small. Will use for future child.   

                               text  
0               great product price  
5       waaay small use futur child  
6            stay vibrant many wash  
8  son really like pink one nervous  
9      waaay small use future child

In [12]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')

# tmp = df['text']

# vectorizer.fit(tmp)

# tmp_tfidf = vectorizer.transform(tmp)

#### Clean the amazon data.

In [13]:
# Clean the amazon text.
amazon = pd.read_csv('~/Downloads/GroupProject/AMAZON.csv')

In [15]:
amazon.head()

overall  verified   reviewTime      reviewerID        asin  \
0        1      True  05 11, 2016  A2E4NSDWJDXHYL  B00ZWWMHTI   
1        2      True  08 23, 2016  A1RZ7GEW807WFX  B01B298Q0S   
2        5      True  06 29, 2014  A1NWDBD77LQ3MB  B00104KJ4C   
3        5      True  02 15, 2014  A1487V05K2FBK4  B000X9FV5M   
4        5      True   03 4, 2014  A2N8ZEA5I6TOPA  B00GV4V8XC   

                                               style  reviewerName  \
0                                                NaN         M. O.   
1                         {'Platform:': ' Xbox One'}  Valerie Reed   
2                         {'Format:': ' Video Game'}             X   
3                         {'Format:': ' Video Game'}  Roger Huston   
4  {'Edition:': ' Standard', 'Platform:': ' Ninte...     Daniel G.   

                                          reviewText  \
0  Did not fit properly no matter what I did. Eve...   
1                                 Meh. Underwhelming   
2  I love it and world recommend this game to my ...   
3  Far Cry 2 is a very enjoyable FPS.  The immens...   
4  There is much to like about the game, even if ...   

                             summary  unixReviewTime vote  \timage  \
0                        Not for me.      1462924800  NaN      NaN   
1                          Two Stars      1471910400  NaN      NaN   
2  Arrived Fast & Got What I Order~!      1404000000  NaN      NaN   
3                 Fun, fun, fun game      1392422400  NaN      NaN   
4                       Awesome game      1393891200  NaN      NaN   

             Category  
0  Video_Games_5.json  
1  Video_Games_5.json  
2  Video_Games_5.json  
3  Video_Games_5.json  
4  Video_Games_5.json

In [21]:
clean_data = clean_df(amazon)

In [23]:
clean_data.to_csv(r'~/Documents/ECS171/reviewClassifier/clean_data.csv')

In [24]:
clean_data

overall  verified   reviewTime      reviewerID        asin  \
0            1      True  05 11, 2016  A2E4NSDWJDXHYL  B00ZWWMHTI   
1            2      True  08 23, 2016  A1RZ7GEW807WFX  B01B298Q0S   
2            5      True  06 29, 2014  A1NWDBD77LQ3MB  B00104KJ4C   
3            5      True  02 15, 2014  A1487V05K2FBK4  B000X9FV5M   
4            5      True   03 4, 2014  A2N8ZEA5I6TOPA  B00GV4V8XC   
5            5      True  01 13, 2017  A1Z3NP8LJB8PQO  B003YMMGEE   
6            5      True  07 14, 2014  A1Z7TAQGXOC4QL  B00109KMOO   
7            5      True  03 23, 2015  A2RODTBQJX7QM7  B00BAWXCP2   
8            4     False  08 24, 2006  A1K31NF81TS0CO  B0002KMHD6   
9            5     False   07 4, 2002  A3ER878Z9WVNRO  B0000631WO   
10           5      True  12 18, 2017  A2BO3O5UVQ8UG3  B00A878J4E   
11           5      True  06 21, 2014  A2BJURUTPLM7ZW  B001EQPDO2   
12           5      True  09 22, 2015   AZTJX0Y6JJKL9  B000VTQ3LU   
13           4      True  02 23, 2013  A3LTK1JJA7G052  B007LQG9A6   
14           2     False  05 14, 2008  A2PT1M0S0S3AOP  B0001Y7404   
15           5      True  06 29, 2017  A2A3BQDXV82PZX  B006JKASCK   
16           5      True  04 17, 2017   ACG14SVOMCMG1  B01EJ9DMQQ   
17           5      True  09 23, 2015   AUHNOD6W2GI1Q  B002BSC5CU   
18           5      True  08 19, 2014  A2B1OM6AN53PKQ  B006ZTHFPS   
19           1     False  07 10, 2007  A1B124AVX0QH2B  B000B43OY4   
20           4      True  05 10, 2013   AXVZA3CQYLQB5  B000SBR9JQ   
21           5     False  04 13, 2016   AW7QUVVFVABUU  B00C7103JI   
22           3      True  09 27, 2008  A1D2QG6A2RI8VN  B00127VE2S   
23           5      True  04 14, 2013  A2OSJWDJWPH3VU  B00ARCWQPS   
24           4      True   12 4, 2015   AWXWYHD0T3O9B  B002I096Q4   
25           5      True  08 15, 2015  A2RZU6WMU5ZS2U  B0012N94WO   
26           5      True  09 12, 2014  A21B8AV7E3MPXE  B0050SW2NA   
27           5      True  01 18, 2016   A3UC5F1WUR1X8  B00L59D9HG   
28           5      True  08 19, 2015  A1F41PH89SL6JB  B00EC6W1O6   
29           4      True   05 9, 2016  A2UKONYF8II9SY  B000P297F2   
...        ...       ...          ...             ...         ...   
14470        1      True  04 30, 2012  A2Y7BUPYLEZ6PE  0007230206   
14471        2      True   08 6, 2009  A2C0ALI7MLHLJZ  0007203136   
14472        2      True   02 1, 2017  A35G8XLKENLP7J  0007155662   
14473        5      True  09 14, 2007  A3KEZLJ59C1JVH  0006551807   
14474        5      True  02 21, 2013   ABPEBP0UWFUK2  0006064922   
14475        5      True   12 1, 2013  A2REG1E0F7XXJL  0004704746   
14476        1      True  08 18, 2016   AMB4EDY6W3AKG  0002256649   
14477        2     False  05 26, 2015  A1XE2JOZZKMEG6  0006172768   
14478        5     False  12 16, 2005  A2YM3KILJ1G0YJ  0001720295   
14479        5     False   03 6, 2015  A2MIOICG0DSFP1  0002247437   
14480        5     False   04 6, 2003  A1TV8O1XY9HREW  0001844423   
14481        5      True  04 10, 2015  A3LPE4KR3HOU6E  0001048767   
14482        5      True  04 12, 2018  A25ET3S2F77G0K  0006280544   
14483        4     False  10 28, 2015  A35R2NR30DMZXV  0007271239   
14484        5      True  01 22, 2016  A1TYREQ3RW7QKP  0001048767   
14485        5      True  07 23, 2001  A24AGAOVQ9V1AT  0007155662   
14486        3      True  07 19, 2014  A13B1RL2P1CKMP  0007215053   
14487        5      True   01 2, 2015  A2BA1V80XPVO4P  0006064922   
14488        5      True  07 15, 2013  A329BKRM2USZ8T  000100039X   
14489        5      True  09 16, 2017  A253E4EDXHUPF9  0006367992   
14490        5      True  07 26, 2011  A1NY3WJ1L8CD94  0002247437   
14491        3      True  04 24, 2010  A3TMZZM7NX8SR9  0007242476   
14492        5      True  01 13, 2017  A3J5OJ1NLYWD00  0001945424   
14493        5     False  02 21, 2010  A2ANWGPM3M2JPD  0001844423   
14494        5      True  04 25, 2013  A1Q4H73Z61HA90  000711835X   
14495        5      True  09 14, 2009  A3LKQQ63SU32ZY  0007173148   
14496